In [76]:
import spacy
from spacy.symbols import *
import numpy as np
from word2number import w2n

# Load trained nlp model
# trained_model_path = "I:\\My Drive\\UCI\\Winter 2023\\COMPSCI 175\\text_parse\\text_parse\models\\trained-trf-pos-model"
nlp = spacy.load ("en_core_web_trf")

from gensim.models import KeyedVectors
from gensim import models
from gensim.models import Word2Vec
import json
# import AstarSearch
import math
import time

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
#model = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, norm_only=True) -> Deprecated

# vector_model_path = "C:\\Users\\Unkow\\Downloads\\GoogleNews-vectors-negative300.bin.gz"
# model = gensim.models.KeyedVectors.load_word2vec_format (vector_model_path, binary=True) # without *norm_only* param

# model.init_sims(replace=True)
# model.save("models/GoogleNews")

# Load saved vector model
# saved_vector_model_path = "models/GoogleNews"
saved_vector_model_path = "I:\\My Drive\\UCI\\Winter 2023\\COMPSCI 175\\text_parse\\text_parse\\models\\GoogleNews"
model = KeyedVectors.load (saved_vector_model_path, mmap='r')

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)  # suppress TracerWarning


In [77]:
DEBUG = False

command_map = {
            "move": {
                "stop": ["move 0", "strafe 0"],
                "forward": "move 1", "forwards": "move 1", "back": "move -1", "backward": "move -1", "backwards": "move -1", "right": "strafe 1", "left": "strafe -1",
                "north": "movenorth 1", "south": "movesouth 1", "east": "moveeast 1", "west": "movewest 1",
                "to": "OBJECT"
            },
            "strafe": {"right": "strafe 1", "left": "strafe -1", "stop": ["strafe 0"]},
            "turn": {"right": "turn 1", "left": "turn -1", "stop": ["turn 0", "pitch 0"],
                     "up": "pitch -1", "down": "pitch 1"},
            "look": {"up": "look -1", "down": "look 1"},
            "pitch": {"up": "pitch -1", "down": "pitch 1", "stop": ["pitch 0"]},
            "jump": {
                "": "jump 1", "up": "jump 1", "stop": ["jump 0"], "forward": "jumpmove 1", "back": "jumpmove -1",
                "backwards": "jumpmove -1", "right": "jumpstrafe 1", "left": "jumpstrafe -1",
                "north": "jumpnorth 1", "south": "jumpsouth 1", "east": "jumpeast 1", "west": "jumpwest 1"
            },
            "crouch": {"": "crouch 1", "stop": ["crouch 0"]},
            "attack": {"": "attack 1", "stop": ["attack 0"]},
            "use": {},
            "get": {"": "OBJECT"},
            "discard": {"": "discardCurrentItem"},
            "stop": ["move 0", "jump 0", "turn 0", "strafe 0", "pitch 0", "crouch 0", "attack 0"],
            "quit": {"": "quit"}
        }

def word_similarity_score (word1, word2):
    score = model.similarity (word1, word2)
    return score

def get_best_match (word, commands_map, threshold):
    scores = []
    for commands in commands_map:
        score = word_similarity_score (word, commands)
        scores.append (score)
    keys = list (commands_map.keys())
    command = keys[np.argmax (scores)]
    return command

def get_similar_command (verb, commands_map, agent_host):
    lemma_verb = verb.lemma_
    # synonyms?
    # left
    if verb.pos != VERB:
        return verb
    # get synonym?
    t = 0.5
    command = get_best_match (lemma_verb, commands_map, t)
    return command

def send_command (verb, commands_map, agent_host):
    if str (verb) == "stop":
        return send_stop_command (commands_map, agent_host)
    command = commands_map.get (str (verb)).get('')
    return [command]
 
def send_command_option (verb, option, commands_map, agent_host):
    for a in option.ancestors:
        if a.pos == VERB:
            for r in a.rights:
                if r.pos == NOUN:
                    return None
                elif r.lemma_ != option.lemma_:
                    break
    
    for l in option.lefts:
        if l.pos == NOUN:
            for l in l.lefts:
                if l.pos == NUM:
                    command = commands_map.get (verb).get (option.lemma_)
                    n  = w2n.word_to_num (l.lemma_)
                    commands = []
                    for i in range (n):
                        commands.append (command)
                    return commands
        else:
            break

    command = commands_map.get (verb).get (option.lemma_)
    return [command]

def send_prop_command (verb, prep, commands_map, agent_host):
    if prep.lemma_ in commands_map.get ("move"):
        for r in prep.rights:
            if r.pos_ == "NOUN":
                # move to OBJECT
                if agent_host == None:
                    return [str (verb) + " " + str (prep) + " " + str(r)]
                AstarSearch.move_to (agent_host, str (r))
                return [None]
            elif r.lemma_ in commands_map.get ("move"):
                c = send_command_option (verb, r, commands_map, agent_host)
                if c:
                    return c

def send_object_command (verb, object, commands_map, agent_host):
    for l in object.lefts:
        if l.pos == NUM:  
            for a in l.ancestors:
                if a.pos == VERB:
                    for r in a.rights:
                        if r.pos == ADV:
                            command = commands_map.get (verb).get (r.lemma_)
                            n  = w2n.word_to_num (l.lemma_)
                            commands = []
                            for i in range (n):
                                commands.append (command)
                            return commands

    if agent_host == None:
        return [str (verb) + " " + str (object)]
    
    if verb == "get":
        AstarSearch.move_to (agent_host, str (object))
        return [None]

    if verb == "use":
        world_state = agent_host.getWorldState()
        if world_state.number_of_observations_since_last_state > 0:
            msg = world_state.observations[-1].text
            observations = json.loads(msg)
            if "inventory" in observations:
                items = observations["inventory"]
                for i in items:
                    type = i["type"]
                    index = i["index"]
                    if object.text in type:
                        return ["hotbar." + str (index + 1) + " 1"]
    elif verb == "attack":
        return [str (verb) + " " + str (object)]
    elif verb == "grab":
        return [str (verb) + " " + str (object)]

def send_stop_command (commands_map, agent_host):
    command = commands_map.get ("stop")
    return command

def parse_root_verb (verb, commands_map, agent_host):
    malmo_command = get_similar_command (verb, commands_map, agent_host)
    
    if DEBUG:
        print ("malmo command: ", malmo_command)

    commands = []
    if sum (1 for r in verb.rights) == 0:
        c = send_command (malmo_command, commands_map, agent_host)
        if c:
            commands.append (c)
    
    for word in verb.rights:
        if DEBUG:
            print ("word: ", word, word.pos_)
        if word.pos == ADV:
            # move forward
            # move backwards
            if word.lemma_ in commands_map.get (malmo_command):
                c = send_command_option (malmo_command, word, commands_map, agent_host)
                if c:
                    commands.append (c)             
        elif word.pos == VERB:
            # subsequent command
            # move forward and dig
            if word.lemma_ == "stop":
                c = send_stop_command (commands_map, agent_host)
                if c:
                    commands.append (c)
            else:
                c = parse_root_verb (word, commands_map, agent_host)
                if c:
                    for c in c:
                        commands.append (c)           
        elif word.pos == ADP:
            # preposition object
            # move to the left
            # move to the right
            # move to
            c = send_prop_command (malmo_command, word, commands_map, agent_host)
            if c:
                commands.append (c)        
        elif word.pos == NOUN or word.pos == PROPN:
            # move 1 block forward
            c = send_object_command (malmo_command, word, commands_map, agent_host)
            if c:
                commands.append (c)
        elif word.pos == CCONJ:
            c = send_command (malmo_command, commands_map, agent_host)
            if c:
                commands.append (c)                

    return commands

def check_agent_pos (agent_host):
        # check to keep agent positioned correctly before a discrete move command
    agent_location = AstarSearch.find_agent_location(agent_host)
    if agent_location:
        if agent_location[0] % 1 != 0.5:
            agent_host.sendCommand(f"tpx {math.floor (agent_location[0]) + 0.5}")
        if agent_location[2] % 1 != 0.5:
            agent_host.sendCommand(f"tpz {math.floor (agent_location[2]) + 0.5}")


def parse_string_command (string, commands_map = command_map, agent_host = None):
    doc = nlp (string)
    commands = []
    for sentence in doc.sents:
            r = sentence.root
            c = parse_root_verb (r, commands_map, agent_host)
            if c:
                for c in c:
                    commands.append (c)
    
    if agent_host == None:
        return commands
    
    for c in commands:
        if c:
            print (c, len (c))
            for c in c:
                if c:
                    check_agent_pos (agent_host)
                    agent_host.sendCommand (c)
            time.sleep(1)

# if __name__ == "__main__":
#     command = input (": ")
#     while command.lower() != "quit":
#         commands = parse_string_command (command, command_map)
        
#         for c in commands:
#             print (c)
        
#         command = input (": ")

In [78]:
command = "move forward five steps"
commands = parse_string_command (nlp (command), command_map)
for c in commands:
    print (c, len (c))

['move 1', 'move 1', 'move 1', 'move 1', 'move 1'] 5


In [79]:
command = "move left five steps"
commands = parse_string_command (nlp (command), command_map)
for c in commands:
    print (c, len (c))

['strafe -1', 'strafe -1', 'strafe -1', 'strafe -1', 'strafe -1'] 5


In [80]:
command = "move right five steps"
commands = parse_string_command (nlp (command), command_map)
for c in commands:
    print (c, len (c))

['strafe 1', 'strafe 1', 'strafe 1', 'strafe 1', 'strafe 1'] 5


In [81]:
command = "move forward three steps and then move backwards three steps"
commands = parse_string_command (command, command_map)
for c in commands:
    print (c, len (c))

['move 1', 'move 1', 'move 1'] 3
[None] 1
['move -1', 'move -1', 'move -1'] 3


In [82]:
command = "move backwards four steps"
commands = parse_string_command (nlp (command), command_map)
for c in commands:
    print (c, len (c))

['move -1', 'move -1', 'move -1', 'move -1'] 4


In [83]:
command = "jump and then move five steps forwards"
commands = parse_string_command (nlp (command), command_map)
for c in commands:
    print (c, len (c))

['jump 1'] 1
['move 1', 'move 1', 'move 1', 'move 1', 'move 1'] 5


In [84]:
command = "move backwards two steps and then turn left and then move forward three steps"
commands = parse_string_command (command, command_map)
for c in commands:
    print (c, len (c))

['move -1', 'move -1'] 2
[None] 1
['turn -1'] 1
[None] 1
['move 1', 'move 1', 'move 1'] 3


In [85]:
command = "crouch and then move left and then stop and then move right"
commands = parse_string_command (command, command_map)
for c in commands:
    print (c, len (c))

['crouch 1'] 1
['strafe -1'] 1
[None] 1
['move 0', 'jump 0', 'turn 0', 'strafe 0', 'pitch 0', 'crouch 0', 'attack 0'] 7


In [86]:
command = "equip the pickaxe"
commands = parse_string_command (command, command_map)
for c in commands:
    print (c, len (c))

['use pickaxe'] 1


In [87]:
p = "diamond_axe"
p1 = "axe".capitalize()

if p1 in p:
    print (p1)

In [88]:
command = "find the pig"
commands = parse_string_command (command, command_map)
for c in commands:
    print (c, len (c))

['get pig'] 1


In [89]:
from spacy import displacy
command = "move to the pig"
displacy.render (nlp (command), style = "dep")

In [90]:
command = "move to the pig"
commands = parse_string_command (command, command_map)
for c in commands:
    print (c, len (c))

['move to pig'] 1


In [91]:
command = "attack and then move forward two steps"
commands = parse_string_command (command, command_map)
for c in commands:
    print (c, len (c))

['attack 1'] 1
['move 1', 'move 1'] 2


In [92]:
words = {
    "move": ["go", "step", "walk"],
    "strafe": ["sidestep"],
    "turn": ["look", "steer", "redirect"],
    "pitch": ["view", "look"],
    "jump": ["leap", "climb"],
    "crouch": ["kneel", "squat", "duck"],
    "attack": ["strike", "hit"],
    "use": ["equip"],
    "get": ["find", "discover", "locat"],
    "discard": ["drop", "throw", "dispose", "remove"],
    "stop": ["halt", "cease"],
    "quit": ["exit", "leave", "withdraw", "abandon"]
}

padding = 10
print (f"{'Word':<{padding}} {'Word':<{padding}} {'Similarity':<{padding}}")
for word in words:
    for w in words[word]:
        similarity = word_similarity_score (word, w)
        print (f"{word: <{padding}} {w: <{padding}} {similarity: <{padding}}")

Word       Word       Similarity
move       go         0.4947884976863861
move       step       0.5192893743515015
move       walk       0.3199070692062378
strafe     sidestep   0.17268125712871552
turn       look       0.3405826985836029
turn       steer      0.4563356339931488
turn       redirect   0.4376186430454254
pitch      view       0.02832110971212387
pitch      look       0.11063489317893982
jump       leap       0.6579698920249939
jump       climb      0.5455771088600159
crouch     kneel      0.4637507200241089
crouch     squat      0.45443809032440186
crouch     duck       0.24699382483959198
attack     strike     0.4166296422481537
attack     hit        0.20998208224773407
use        equip      0.2946905195713043
get        find       0.5191866755485535
get        discover   0.28421375155448914
get        locat      0.08558519184589386
discard    drop       0.22030499577522278
discard    throw      0.29430434107780457
discard    dispose    0.5484936237335205
discard    rem

In [93]:
w = command_map.get("ppp")
if w == None:
    print ("none")

none


In [94]:
padding = 20

words = ["strike"]
for w in words:
    print ("Word: ", w)
    for c in command_map:
        similarity = word_similarity_score (w, c)
        print (f"{c + ':': <{padding}} {similarity: <{padding}}")
    print()

Word:  strike
move:                0.21112419664859772 
strafe:              0.12763535976409912 
turn:                0.14728482067584991 
look:                0.04816334322094917 
pitch:               0.252928763628006   
jump:                0.1185469999909401  
crouch:              0.18934939801692963 
attack:              0.4166296422481537  
use:                 0.04601665213704109 
get:                 0.14010487496852875 
discard:             0.05633421987295151 
stop:                0.13084031641483307 
quit:                0.15927809476852417 



In [95]:
nlp_lg = spacy.load ("en_core_web_lg")

In [96]:
words = ["strike"]
for w in words:
    print ("Word: ", w)
    for c in command_map:
        similarity = nlp_lg (w).similarity (nlp_lg (c))
        print (f"{c + ':': <{padding}} {similarity: <{padding}}")
    print()    

Word:  strike
move:                0.45278189120395546 
strafe:              0.42521033366502003 
turn:                0.39054223557421763 
look:                0.16421402020221976 
pitch:               0.3704094655891882  
jump:                0.28651761712600193 
crouch:              0.11708493286142005 
attack:              0.6477336955049335  
use:                 0.14544984748344783 
get:                 0.2387346415970668  
discard:             0.21439720844750057 
stop:                0.29185042409280737 
quit:                0.16667880411776206 



In [97]:
model.most_similar ("attack", topn = 30)

[('attacks', 0.7638030648231506),
 ('attck', 0.606964111328125),
 ('atack', 0.6036049723625183),
 ('assault', 0.5970097780227661),
 ('atttack', 0.5820074081420898),
 ('bombing', 0.5777696371078491),
 ('assaults', 0.5646901726722717),
 ('attacking', 0.5615357160568237),
 ('attacked', 0.5553597211837769),
 ('counterattack', 0.5205312967300415),
 ('unprovoked_attack', 0.5182728171348572),
 ('atacks', 0.5176907181739807),
 ('suicide_bombing', 0.5143910646438599),
 ('ambush', 0.5107647180557251),
 ('atttacks', 0.5064945220947266),
 ('raid', 0.502953290939331),
 ('blasts', 0.49935752153396606),
 ('attack.The', 0.49905073642730713),
 ('vicious_unprovoked', 0.49792152643203735),
 ('bombings', 0.4972839653491974),
 ('onslaught', 0.4892962872982025),
 ('attackers', 0.48471570014953613),
 ('Attack', 0.48243942856788635),
 ('northwestern_Hurriyah_neighborhood', 0.4810367524623871),
 ('theattack', 0.4677920937538147),
 ('incursion', 0.4648672044277191),
 ('bomb_blast', 0.4632481336593628),
 ('count

In [98]:
from spacy import displacy

In [99]:
command = "move right"
displacy.render (nlp (command), style = "dep")